In [5]:
import os
import traceback
import logging
import os.path
import networkx as nx
import pandas as pd
import numpy as np
import pylab as pl
from matplotlib import collections  as mc
import glob
import re
import csv
import shutil

In [6]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception as e:
    logging.error(traceback.format_exc())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# simversion = '_v01'
simversion = '_plasmid_v01'
simversionPangraph = '_assemBiGraph2_v01' 
# 1 True, 0 False
run_prokka = 0
run_art = 0
run_spades = 0
run_panta = 0
split_paralogs = 1
run_multicsar = 0
run_ragout = 0
run_pangraph = 1
art_simulator = 0
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
# pangenome_data = '/data/hoan/amromics/data/ncbi/Ecoli50random/'
pangenome_data = '/data/hoan/amromics/data/ncbi/Kp50random/'
prokka_target_dir = '/data/hoan/amromics/data/ncbi/Ecolitest/prokka/'
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [8]:
if 'Kp' in pangenome_data:
    data_base = ' --genus Klebsiella --species pneumoniae --cpus 30'
elif 'Ecoli' in pangenome_data:
    data_base = ' --genus Escherichia --species coli --cpus 30'
else:
    data_base = ' --cpus 30'
print(data_base,data_base,data_base,data_base)
print(data_base,data_base,data_base,data_base)
output_dir = pangenome_data + 'temp'
prokka_dir = pangenome_data + 'prokka'
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && '

if run_prokka:
    fffile = pangenome_data + 'fna/*.fna'
    list_database = glob.glob(fffile)
    for i in range(len(list_database)):
        input_data = list_database[i]
        print(input_data)
        file_name = input_data.split("/")[-1][:-4]
        prokka_bin = conda_dir + 'prokka --outdir ' + output_dir +data_base +' --prefix ' + file_name +' '+input_data
        os.system(prokka_bin)
        os.system('cp ' + output_dir + '/'+file_name+'.gff ' + prokka_dir)
        os.system('rm -r '+ output_dir)

 --genus Klebsiella --species pneumoniae --cpus 30  --genus Klebsiella --species pneumoniae --cpus 30  --genus Klebsiella --species pneumoniae --cpus 30  --genus Klebsiella --species pneumoniae --cpus 30
 --genus Klebsiella --species pneumoniae --cpus 30  --genus Klebsiella --species pneumoniae --cpus 30  --genus Klebsiella --species pneumoniae --cpus 30  --genus Klebsiella --species pneumoniae --cpus 30


In [ ]:
if 1:
    os.system('rm -r ' + pangenome_data + 'fna/')
    os.system('mkdir ' + pangenome_data + 'fna/')
    os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
    os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')

In [ ]:
# ### Run ART simulator
# art_simulator = 1
# sim_dir = '/data/hoan/amromics/simulation/'
# art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002899495.1_ASM289949v1_genomic.fna'      # test 2
# sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13]
# sim_output = '/data/hoan/amromics/simulation/art_output/sim_reads_' + sim_name + simversion[-4:]
# if run_art: 
#     if simversion[-3:]=='v01':
#         os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 60 -o '+sim_output) # _v01
#     if simversion[-3:]=='v02':
#         os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 10 -o '+sim_output) # _v02
#     if simversion[-3:]=='v03':
#         os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v03
#     if simversion[-3:]=='v04':
#         os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 10 -o '+sim_output) # _v04

### Run SPADES assembly

In [10]:
# ref_data = "/data/hoan/amromics/data/ncbi/Kp_BAA-2146/refSeq/GCF_000364385.3_ASM36438v3_genomic.fna"
# read_input = '/data/hoan/amromics/data/ncbi/Kp_BAA-2146/reads_data/ERR1474547_'

# ref_data = "/data/hoan/amromics/data/ncbi/paired_data/ecolik12/GCF_000005845.2_ASM584v2_genomic.fna"
# read_input = '/data/hoan/amromics/data/ncbi/paired_data/ecolik12/SRR001665_'

# ref_data = "/data/hoan/amromics/data/ncbi/paired_data/ecoli_CFSAN027343/GCF_004358405.1_ASM435840v1_genomic.fna"
# read_input = '/data/hoan/amromics/data/ncbi/paired_data/ecoli_CFSAN027343/SRR8333591_'

# ref_data = "/data/hoan/amromics/data/ncbi/paired_data/ecoli_CFSAN027350/GCF_004358385.1_ASM435838v1_genomic.fna"
# read_input = '/data/hoan/amromics/data/ncbi/paired_data/ecoli_CFSAN027350/SRR8333590_'

ref_data = "/data/hoan/amromics/data/ncbi/paired_data/kq_700603/GCF_001596075.2_ASM159607v2_genomic.fna"
read_input = '/data/hoan/amromics/data/ncbi/paired_data/kq_700603/ERR1474548_'

print("Ref data: ", ref_data)
if art_simulator:
    sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13]
else:
    sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13] +'_'+ read_input.split("/")[-1]
print(read_input)

Ref data:  /data/hoan/amromics/data/ncbi/paired_data/kq_700603/GCF_001596075.2_ASM159607v2_genomic.fna
/data/hoan/amromics/data/ncbi/paired_data/kq_700603/ERR1474548_


In [11]:
sim_name

'kq_700603_GCF_001596075_ERR1474548_'

In [12]:
# spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output_' + sim_name + simversion[-4:]
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output_' + sim_name + simversion[-4:]

In [13]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    # os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)
    os.system(spades_bin+' --isolate -1 '+read_input+'1.fastq -2 '+read_input+'2.fastq -t 50 -o '+spades_output)

In [14]:
spades_output

'/data/hoan/amromics/simulation/art_output/spades_output_kq_700603_GCF_001596075_ERR1474548__v01'

### Run panta

In [15]:
simversionRef = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversion
simversionRef

'_Kp50random_kq_700603_GCF_001596075_ERR1474548__plasmid_v01'

In [16]:
spades_output

'/data/hoan/amromics/simulation/art_output/spades_output_kq_700603_GCF_001596075_ERR1474548__v01'

## Run this for the first time

In [ ]:
if 1:
    conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && '
    input_data = spades_output + '/contigs.fasta'
    os.system('rm -r ' + prokka_target_dir +'temp/')
    prokka_bin = conda_dir + 'prokka --outdir ' + prokka_target_dir +'temp' +data_base +' --prefix ' + sim_name +' '+input_data
    os.system(prokka_bin)
    os.system('cp ' + prokka_target_dir +'temp/'+sim_name+'.gff ' + prokka_target_dir + 'g1.gff')
    os.system('cp ' + prokka_target_dir +'temp/'+sim_name+'.gff ' + prokka_target_dir + sim_name+'.gff')
    os.system('rm -r ' + prokka_target_dir +'temp/')
    print(prokka_bin)

In [ ]:
from pangraph.utils import export_metadata

In [17]:
pantaconda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =pantaconda_dir+'python /data/hoan/amromics/panta_memopt_v2/panta/pan-genome.py'
# panta_bin =pantaconda_dir+'python /data/hoan/amromics/panta_memopt/pan-genome.py'
panta_output = '/data/hoan/amromics/genome-graph/scaffold_output/pantaOut/pantaOut' + simversionRef
# panta_output_temp = panta_output + '_temp'
panta_input = pangenome_data +'prokka/*.gff'
if run_panta:
    os.system('rm -r '+panta_output)
    # os.system('rm -r '+panta_output_temp)
    # ## run all samples + new samples
    # # os.system('rm -r '+panta_output)
    # # os.system('rm -r '+panta_output_temp)
    # # os.system('cp '+prokka_target_dir + sim_name+'.gff ' + pangenome_data +'prokka/')
    # # # cmd_panta = panta_bin + ' main -o ' + panta_output +' -g ' + panta_input + ' -i 0.7'
    # # cmd_panta = panta_bin + ' main -o ' + panta_output +' -g ' + panta_input 
    # # print(cmd_panta)
    # # os.system(cmd_panta)
    # export_metadata(panta_output)     
   
    ## Split into 2 steps
    # os.system('cp '+prokka_target_dir + sim_name+'.gff ' + pangenome_data +'prokka/')
    cmd_panta = panta_bin + ' main -o ' + panta_output +' -g ' + panta_input
    print(cmd_panta)
    os.system(cmd_panta)
    
    ## Copy to temp directory
   # os.system('rm -r '+panta_output_temp)
   # os.system('cp -R '+ panta_output + ' ' + panta_output_temp)
    cmd_panta = panta_bin + ' add -c ' + panta_output +' -g ' + prokka_target_dir + sim_name+'.gff -s'
    print(cmd_panta)
    os.system(cmd_panta)
    export_metadata(panta_output)

In [ ]:
# # file_name= ref_data.split("/")[-1][:-4]
# # cmd_copy = 'cp ' + prokka_target_dir + file_name + '.gff ' + prokka_target_dir + 'g1.gff'
# # panta_target_input = prokka_target_dir + file_name + '.gff'
# # os.system(cmd_copy)
# # panta_target_input = prokka_target_dir + 'g1.gff'
# ### Khong the lam nhu the nay duoc vi day co phai la cai ta can noi dau???
# # panta_target_input = prokka_target_dir + 'g1.gff'
# panta_target_input = prokka_target_dir + sim_name + '.gff'
# cmd_panta = panta_bin + ' add -c ' + panta_output +' -g ' + panta_target_input
# if run_panta:
#     os.system(cmd_panta)
#     # create samples.tsv
#     from pangraph.utils import export_metadata
#     export_metadata(panta_output)

In [ ]:
os.system('rm '+ pangenome_data +'prokka/' + sim_name+'.gff')

### Run pangraph

In [19]:
from pangraph import PanGraph

In [20]:
# set parameters
data_dir = panta_output 
# incomplete_sample_name = 'g1'
incomplete_sample_name = sim_name
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [21]:
simversionRefNew = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversionPangraph
# pangraph_output_greedy_default = scaffold_out_dir + 'pangraph_greedy'+ simversionRef + '.fasta'
# pangraph_output_opt_default = scaffold_out_dir + 'pangraph_opt'+ simversionRef + '.fasta'
pangraph_output_sensitive = scaffold_out_dir + 'pangraph_sensitive'+ simversionRefNew + '.fasta'
pangraph_output_default = scaffold_out_dir + 'pangraph_default'+ simversionRefNew + '.fasta'
pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversionRefNew + '.fasta'

In [22]:
# panta_output_temp

In [23]:
incomplete_sample_name

'kq_700603_GCF_001596075_ERR1474548_'

In [ ]:
PASAversion = '_BiGR0.3'; MLR = 0; SInfer = 1; min_weight_val = 0.3; 
simversionRefNew = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + PASAversion 
if run_pangraph:
    print(PASAversion, '------------------------------------------------------------------')
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_sensitive, maximum_matching, MLR, SInfer, min_weight_val)
    except Exception as e:
        logging.error(traceback.format_exc())

In [ ]:
PASAversion = '_BiGR0.3'; MLR = 0; SInfer = 0; min_weight_val = 0.3; 
simversionRefNew = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + PASAversion 
if run_pangraph:
    print(PASAversion, '------------------------------------------------------------------')
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'optimal'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching, MLR, SInfer, min_weight_val)
    except Exception as e:
        logging.error(traceback.format_exc())

_BiGR0.3 ------------------------------------------------------------------
Should we use target sequence as reference, hence, the strand will be ok
Set minimum on number of nucleotides =  10 NUMBER OF COMPUTED CONTIGS: 3575
Clip the matrix 0.2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Use modified  assembly graph
Use union graph: overlap + assembly: done
Re-infer the contigs strand: NO


In [ ]:
PASAversion = '_MLRInfer1.0'; MLR = 1; SInfer = 1; min_weight_val = 1.0; 
# simversionRefNew = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + PASAversion 
# pangraph_output_greedy = scaffold_out_dir + 'pangraph_greedy'+ simversionRefNew + '.fasta'
# pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversionRefNew + '.fasta'
if run_pangraph:
    print(PASAversion, '------------------------------------------------------------------')
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_default, maximum_matching, MLR, SInfer, min_weight_val)
    except Exception as e:
        logging.error(traceback.format_exc())

### Run scaffold methods

In [ ]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'

In [ ]:
multicsar_output = scaffold_out_dir + 'multicsar/multicsar' + simversionRef
if run_multicsar:
    os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
    os.system('rm ' + pangenome_data + 'fna/g1*')
    os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
    # os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

In [ ]:
multicsar_output

In [ ]:
### Ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'multicsar/ragout' + simversionRef
if run_ragout:
    # write receipt file
    ref_files_list = glob.glob(pangenome_data+'fna/*')
    n_files = len(ref_files_list)
    receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversionRef+'.rcp'
    f = open(receipt_file_dir, "w")
    f.write('.references = ')
    for idx in range(n_files-1):
        f.write('r' + str(idx) + ',')
    f.write('r'+str(n_files-1)+'\n')
    f.write(".target = mg1655\n\n")
    for idx in range(n_files):
        f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
    f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
    f.close()

In [ ]:
if run_ragout:
    os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
    # ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

In [ ]:
print(ragout_output)

### Quast 

In [ ]:
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversionRefNew
quast_output

In [ ]:
quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
spades_output_fasta = spades_output+'/contigs.fasta'
multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
# if 1:
#     if os.path.isfile(ragout_output_fasta):
#         os.system(quast_bin + pangraph_output_default+' '+ pangraph_output_sensitive+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
#         '-l "Pasa, Pasa_sensitive, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
#     else:
#         os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
#         '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
if 1:
    if os.path.isfile(ragout_output_fasta):
        os.system(quast_bin + pangraph_output_default+' '+ pangraph_output_sensitive+' '+ pangraph_output_opt+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pasa, Pasa_sensitive, Pasa_OPT, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
    else:
        os.system(quast_bin + pangraph_output_default+' '+ pangraph_output_sensitive+' '+ pangraph_output_opt+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pasa, Pasa_sensitive, Pasa_OPT, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

In [ ]:
end

In [ ]:
# pangraph.gene_position
pangraph.sample_info

In [ ]:
sample_df = pangraph.gene_position.loc[pangraph.gene_position["SampleID"]==49]

In [ ]:
aa = sample_df.iloc[0,2].split(';')

In [ ]:
sample_df

In [ ]:
for key in pangraph.strand:
    print(key)

In [ ]:
# # pangraph.H["weight"][0:4]
# iteration = 0
# for u,v,a in pangraph.H.edges(data=True):
#     print(a)
#     iteration += 1
#     if iteration > 10:
#         break;

In [ ]:
# nx.write_gml(pangraph.assembly_graph, quast_output+'/assembly_graph.gml')

In [ ]:
pangraph.assembly_graph.number_of_edges()

In [ ]:
len(list(nx.weakly_connected_components(pangraph.assembly_graph)))

In [ ]:
# pangraph.weighted_CG.nodes()

In [ ]:
len(list(nx.weakly_connected_components(pangraph.H)))

In [ ]:
pangraph.H.number_of_nodes(), pangraph.H.number_of_edges()

In [ ]:
ref_data

## Analysis

In [ ]:
# minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli100random_Ecoli_plasmid_v01.fasta > contigs100vsref_mapping_v01a.paf
cmd_minimap = '~/miniconda3/envs/amromics/bin/minimap2 '+ref_data +' '+ spades_output +'/contigs.fasta' +' > scaffold_output/minimap_out/mapping.paf' 

In [ ]:
os.system(cmd_minimap)
df = pd.read_csv("scaffold_output/minimap_out/mapping.paf", sep ="\t", header=None)

In [ ]:
df
ref_id = df.iloc[0,5]
ref_id

In [ ]:
final_df = df.loc[df[5] == ref_id]
final_df = final_df.sort_values(by=[7])

In [ ]:
target_id = 0
gene_position = pd.read_csv(panta_output + '/gene_position.tsv', delimiter='\t', header=None)
gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']
target_contigs = [gene_position.iloc[i, 1] for i in range(len(gene_position.index)) if gene_position.iloc[i, 0]==target_id]

In [ ]:
samples = pd.read_csv(panta_output + '/samples.tsv', delimiter='\t', header=None)
# samples

In [ ]:
target_contigs_list = [final_df.iloc[i, 0] for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]
target_contigs_sorted = [final_df.iloc[i, 0]+'---'+str(final_df.iloc[i, 7])+'>'+str(final_df.iloc[i, 8]) for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]

In [ ]:
gene_position[gene_position.ContigName=='NODE_95_length_1592_cov_58.378660'].iloc[0,2].split(';')

In [ ]:
cluster_id = pangraph.gene2cluster_dict['Ecolitest_GCF_002860105-NODE_95_length_1592_cov_58.378660-PCILKDFA_04850@-1']

In [ ]:
gene_info = pd.read_csv(panta_output + "/gene_info.tsv", delimiter='\t', header=None)
gene_info.columns =['GeneName', 'SampleID', 'clusterID']

In [ ]:
gene_info[gene_info.clusterID==cluster_id]

In [ ]:
target_contigs_sorted[-4:]

In [ ]:
# pangraph.adj_list0

In [ ]:
# cmd_dnadiff = '~/miniconda3/envs/amromics/bin/dnadiff '+ref_data +' '+ spades_output +'/contigs.fasta -p dna_diff/'+sim_name
# os.system(cmd_dnadiff)

In [ ]:
# Don't use SPADES, shovil keep files.
# python readSimulator/readSimulator.py --outdir shredded_reads  --input ncbi-genomes-2023-01-07/GCF_000240185.1_ASM24018v2_genomic.fna --simulator art --simulator_path art_bin_MountRainier/art_illumina --iterations 100 --readlen 100 --depth 100  --opts '-ss HS20 -sam -p -m 400 -s 60'
# investigate why we failed? #1/ connectivity in the assembly graph:
# multiplicity of each node: now 1, change to more: use length and coverage
# assembly graph: change to 500

In [ ]:
def get_node_coverage(node_id):
    return float(node_id.split('_')[5])
get_node_coverage('NODE_45_length_30843_cov_18.832500')
target_contigs_multi_1 = [item for item in target_contigs_list if get_node_coverage(item) <= 25.0]

In [ ]:
def get_value(edge_df0, source_id, target_id):
    # if source_id in edge_df0['source'].values and target_id in edge_df0['target'].values:
    #     print(edge_df0[((edge_df0['source'] == source_id) & (edge_df0['target'] == target_id))])
    # else:
    #     print('No value')
    df_res = edge_df0[((edge_df0['source'] == source_id) & (edge_df0['target'] == target_id))]
    if len(df_res.index) == 1:
        print(df_res.iloc[0, 2])
    else:
        print("No value")
# pangraph.edge_df0

def next_node_multi_1(target_contigs_list, idx):
    ## find the next node of multiplicity 1 after the index: idx
    current_idx = idx + 1
    while(get_node_coverage(target_contigs_list[current_idx]) >= 25.0):
        current_idx += 1
    return target_contigs_list[current_idx]

In [ ]:
# target_contigs_multi_1

In [ ]:
assembly_graph = pangraph.assembly_graph
for i in range(len(target_contigs_list)-1):
    print('---------------------- i =', i, '---------------------------------------------')
    node1 = target_contigs_list[i]
    node2 = target_contigs_list[i+1]
    print(target_contigs_sorted[i],'::::::::', target_contigs_sorted[i+1])
    source_id = 'C-' + str(pangraph.tail_contig[node1])
    target_id = 'C-' + str(pangraph.head_contig[node2]) 
    source_node = node1 + pangraph.strand[node1]
    target_node = node2 + pangraph.strand[node2]
    
    if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
        print("Is there a path in the assembly graph? ", end ='')
        print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
        if nx.has_path(assembly_graph, source_node, target_node):
            path_nucleotides = -1
            path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
            for idxp in range(1, len(path)-1):
                path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
            print(' number of nucleotides: ', path_nucleotides)
        else:
            print('<----xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print("Is there a path in the pangenome graph? ", end ='')
    if nx.has_path(pangraph.H, source_id, target_id):
        p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
        weight_p = 0.0
        for node_p_idx in range(len(p)-1):
            weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
        if len(p) >= 2:
            weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
        print("Yes!, cost = ", weight_p)
    else:
        print("No path in Pangraph")
    print("Real cost between the two contigs: ", end ='')
    get_value(pangraph.edge_df0, node1, node2)
        
#     if get_node_coverage(node2) >= 25.0:
#         print('**************************************************')
#         node2 = next_node_multi_1(target_contigs_list, i+1)
#         print(node1, node2)
#         source_id = 'C-' + str(pangraph.tail_contig[node1])
#         target_id = 'C-' + str(pangraph.head_contig[node2]) 
#         source_node = node1 + pangraph.strand[node1]
#         target_node = node2 + pangraph.strand[node2]
#         get_value(pangraph.edge_df0, node1, node2)
#         if nx.has_path(pangraph.H, source_id, target_id):
#             p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
#             weight_p = 0.0
#             for node_p_idx in range(len(p)-1):
#                 weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
#             if len(p) >= 2:
#                 weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
#             print(weight_p)
#         else:
#             print("No path in Pangraph")
#         if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
#             print("Is there a path in the assembly graph? ", end ='')
#             print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
#             if nx.has_path(assembly_graph, source_node, target_node):
#                 path_nucleotides = -1
#                 path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
#                 for idxp in range(1, len(path)-1):
#                     path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
#                 print("Number of nucleo: ",  path_nucleotides) #len(path)
#             else:
#                 print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
            
    
    #     # sum_len = 0
    #     # if nx.has_path(pangraph.H, source_id, target_id) or nx.has_path(assembly_graph, source_node, target_node):
    #     #     sum_len += int(final_df.iloc[i,1])
    #     #     print(sum_len, end='')
    # else:
    #     print("Restart............................................")
    #     sum_len = 0
    # # print('')

In [ ]:
# pangraph.edge_df0

In [ ]:
# pangraph.edge_df0['source'].values

In [ ]:
## Xet chieu trong trinh dai dien de sua roi inversion.

In [ ]:
# Gui 3 file reference + 1 test sample

In [ ]:
# import glob
# import os
# list_file = glob.glob("/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kp50random_Kptest*/report.pdf")

In [ ]:
# for file in list_file:
#     sample_id = file.split("/")[-2].split("_")[4]
#     cmd = 'mv ' + file +' '+ file[:-10] + str(sample_id) + 't3.pdf'
#     # print(cmd)
#     os.system(cmd)